# 1 - PROBLEM

## Requisitos

1. **Leitura de Dados:** Desenvolva um componente capaz de ler uma lista de objetos contidos em um arquivo JSON, representando informações sobre tema escolhidoes.

2. **Mapeamento, Filtro e Redução:** Implemente funcionalidades para realizar mapeamento, filtragem e redução dos dados, proporcionando uma análise mais refinada das informações contidas no conjunto de dados.

3. **Manipulação de Dados Individuais:** Permita a leitura individual, atualização e exclusão de do arquivossores, mantendo o arquivo JSON sempre atualizado.

4. **Validações de Operações:** Integre validações utilizando blocos try-except e raise para garantir a robustez das operações, prevenindo erros e assegurando a consistência dos dados.

5. **Obtenção de Estatísticas Simples:** Desenvolva uma função para extrair dados estatísticos simples, como média, máximo e mínimo, por exemplo, aum item X nos dados de exatas.

6. **Identificação de Máximos/Mínimos com Detalhes:** Crie uma função que retorne uma lista de tuplas, contendXdo professor e o valor máximo (ou mínimo) de algum atributo numérico. Esta função deve ser configurável para fornecer estatísticas de máximo ou mínimo.

7. **Exportação de Dados Estatísticos para CSV:** Implemente a capacidade de salvar os dados estatísticos obtidos em um arquivo CSV, permitindo uma análise posterior ou compartilhamento fácil dos resultados.

# 2 - IMPORTS

## 2.1 - Bibliotecas

In [1]:
import json

## 2.2 - Funções

### 2.2.1 - Leitura do Arquivo

In [2]:
def carrega_dados(path:str = 'data/receitas.json') -> list[dict]:
    try:
        with open(path, 'r', encoding='utf-8') as arquivo:
            dados = arquivo.read()
            return json.loads(dados)
    except FileNotFoundError:
        return []

### 2.2.2 - Salvamento do Arquivo

In [3]:
def salvar_dados(dados:list[dict], path:str = 'data/receitas.json') -> bool:
    try:
        with open(path, 'w', encoding='utf-8') as arquivo:
            arquivo.write(json.dumps(dados))
            return True
    except Exception:
        return False

### 2.2.3 - Visualização do Arquivo

In [4]:
def formata_lista_receitas(lista_receitas):
    return '\n'.join(map(formata_receita, lista_receitas))

In [5]:
def formata_receita(receita):
    nome = receita['nome']
    ingredientes = formata_ingredientes(receita['ingredientes'])
    instrucoes = receita['instrucoes']
    return f'{nome}:\n Ingredientes:\n  {ingredientes}\n Instruções: {instrucoes}\n'

In [6]:
def formata_ingredientes(ingredientes: list) -> str:
    return '\n  '.join(map(formata_dicionario, ingredientes))

In [7]:
def formata_dicionario(dados: dict) -> str:
    return ' | '.join([f'{k}: {v}' for k, v in dados.items()])

### 2.2.4 - Leitura do Arquivo

In [8]:
def retorna_dicionario(dado: dict, escolha=''):
    return dado if escolha in dado['nome'] else None

In [9]:
def verifica_escolha(data, escolha=''):
    resultado = []
    for dado in data:
        if retorna_dicionario(dado, escolha) is not None:
            resultado.append(dado)
    return resultado

### 2.2.5 - Adiciona Elemento

In [1]:
def obter_nome():
    while True:
        nome = input("Digite o nome da nova receita: ").strip().capitalize()
        if nome:
            return nome
        else:
            print("Nome inválido. Por favor, digite novamente.")

In [2]:
def obter_ingrediente():
    while True:
        ingrediente = input("Digite o nome do ingrediente (ou 'fim' para encerrar): ").strip().capitalize()
        if ingrediente.lower() == 'fim' or ingrediente:
            return ingrediente
        else:
            print("Ingrediente inválido. Por favor, digite novamente.")

In [3]:
def obter_quantidade():
    while True:
        quantidade = input("Digite a quantidade do ingrediente: ").strip()
        if quantidade and validar_numero(quantidade):
            return float(quantidade)
        else:
            print("Quantidade inválida. Por favor, digite novamente.")

In [4]:
def obter_custo():
    while True:
        custo = input("Digite o valor do ingrediente: ").strip()
        if custo and validar_numero(custo):
            return float(custo)
        else:
            print("Custo inválido. Por favor, digite novamente.")

In [5]:
def validar_numero(valor):
    try:
        return float(valor) >= 0
    except ValueError:
        return False

In [6]:
def obter_instrucao():
    return input("Digite a instrução de preparo da receita: ").capitalize()

In [7]:
def criar_nova_receita():
    nova_receita = {'nome': obter_nome(), 'ingredientes': []}

    while True:
        ingrediente = obter_ingrediente()
        if ingrediente.lower() == 'fim':
            break

        quantidade = obter_quantidade()
        custo = obter_custo()

        nova_receita['ingredientes'].append({
            'nome': ingrediente,
            'quantidade': quantidade,
            'custo': custo
        })

    nova_receita['instrucoes'] = obter_instrucao()
    return nova_receita

# 3 - Code Start

## 3.1 - Carregamento de Dados

In [10]:
dados = carrega_dados()

In [11]:
dados

[{'nome': 'Lasanha',
  'ingredientes': [{'ingrediente': 'macarrão para lasanha',
    'quantidade': '200g',
    'custo': '2.50'},
   {'ingrediente': 'molho de tomate', 'quantidade': '500ml', 'custo': '3.00'},
   {'ingrediente': 'carne moída', 'quantidade': '400g', 'custo': '5.00'},
   {'ingrediente': 'queijo', 'quantidade': '200g', 'custo': '4.00'},
   {'ingrediente': 'manteiga', 'quantidade': '50g', 'custo': '1.00'}],
  'instrucoes': 'Cozinhe o macarrão, faça o molho de tomate, cozinhe a carne, monte as camadas e asse no forno.'},
 {'nome': 'Salada Caesar',
  'ingredientes': [{'ingrediente': 'alface romana',
    'quantidade': '1 cabeça',
    'custo': '2.00'},
   {'ingrediente': 'croutons', 'quantidade': '1 xícara', 'custo': '1.50'},
   {'ingrediente': 'queijo parmesão', 'quantidade': '50g', 'custo': '3.00'},
   {'ingrediente': 'peito de frango grelhado',
    'quantidade': '300g',
    'custo': '6.00'},
   {'ingrediente': 'molho Caesar', 'quantidade': '150ml', 'custo': '2.50'}],
  'instr

## 3.2 - Visualizar os dados carregados

In [12]:
resultado_formatado = formata_lista_receitas(dados)

In [13]:
print(resultado_formatado)

Lasanha:
 Ingredientes:
  ingrediente: macarrão para lasanha | quantidade: 200g | custo: 2.50
  ingrediente: molho de tomate | quantidade: 500ml | custo: 3.00
  ingrediente: carne moída | quantidade: 400g | custo: 5.00
  ingrediente: queijo | quantidade: 200g | custo: 4.00
  ingrediente: manteiga | quantidade: 50g | custo: 1.00
 Instruções: Cozinhe o macarrão, faça o molho de tomate, cozinhe a carne, monte as camadas e asse no forno.

Salada Caesar:
 Ingredientes:
  ingrediente: alface romana | quantidade: 1 cabeça | custo: 2.00
  ingrediente: croutons | quantidade: 1 xícara | custo: 1.50
  ingrediente: queijo parmesão | quantidade: 50g | custo: 3.00
  ingrediente: peito de frango grelhado | quantidade: 300g | custo: 6.00
  ingrediente: molho Caesar | quantidade: 150ml | custo: 2.50
 Instruções: Misture os ingredientes, adicione o frango grelhado e regue com molho Caesar.

Sopa de Abóbora:
 Ingredientes:
  ingrediente: abóbora | quantidade: 1 kg | custo: 2.50
  ingrediente: cebola | qu

## 3.3 - Leitura Individual

In [14]:
x = verifica_escolha(dados, 'Salada Caesar')

In [15]:
x

[{'nome': 'Salada Caesar',
  'ingredientes': [{'ingrediente': 'alface romana',
    'quantidade': '1 cabeça',
    'custo': '2.00'},
   {'ingrediente': 'croutons', 'quantidade': '1 xícara', 'custo': '1.50'},
   {'ingrediente': 'queijo parmesão', 'quantidade': '50g', 'custo': '3.00'},
   {'ingrediente': 'peito de frango grelhado',
    'quantidade': '300g',
    'custo': '6.00'},
   {'ingrediente': 'molho Caesar', 'quantidade': '150ml', 'custo': '2.50'}],
  'instrucoes': 'Misture os ingredientes, adicione o frango grelhado e regue com molho Caesar.'}]

## 3.4 - Adiciona Elemento

In [8]:
criar_nova_receita()

Digite o nome da nova receita:  Bolinho de arroz
Digite o nome do ingrediente (ou 'fim' para encerrar):  ovo
Digite a quantidade do ingrediente:  1
Digite o valor do ingrediente:  0.50
Digite o nome do ingrediente (ou 'fim' para encerrar):  fim
Digite a instrução de preparo da receita:  acredita


{'nome': 'Bolinho de arroz',
 'ingredientes': [{'nome': 'Ovo', 'quantidade': 1.0, 'custo': 0.5}],
 'instrucoes': 'Acredita'}